### Process_data_agent_scores.ipynb

This notebook runs through each agent in a file, and finds the decision that is most ideal for that agent and saves this to a file

In [32]:
import pandas as pd

filename = "/home/ia23938/Documents/GitHub/ValueSystemsAggregation/data/ess_example_data/processed_data_ess.csv"
savename = "/home/ia23938/Documents/GitHub/ValueSystemsAggregation/data/ess_example_data/processed_data_ess_with_decisions.csv"

In [33]:
# Compute decision
df = pd.read_csv(filename)
for index, row in df.iterrows():
    df.at[index, 'rel-nonrel'] = row['rel'] / (row['rel'] + row['nonrel'])
    df.at[index, 'nonrel-rel'] = row['nonrel'] / (row['rel'] + row['nonrel'])
    df.at[index, 'decision'] = (df.at[index, 'rel-nonrel'] * row['a_div_rel']) + (df.at[index, 'nonrel-rel'] * row['a_div_nonrel'])

#df.to_csv(savename)

#### Grab consensus value system and store decisions for each relevant consensus

In [34]:
relevent_consensuses =  [1.0, 1.1, 1.5, 4.0, 10.0]
relevent_columns = ['p','Rel-Nonrel', 'Nonrel-Rel', 'Rel_div_p', 'Nonrel_div_p']
preference_filename = "/home/ia23938/Documents/GitHub/ValueSystemsAggregation/data/ess_example_data/23-10-example-preference-results.csv"
actions_filename = "/home/ia23938/Documents/GitHub/ValueSystemsAggregation/data/ess_example_data/23-10-example-action-results.csv"

final_df_savename = "/home/ia23938/Documents/GitHub/ValueSystemsAggregation/data/ess_example_data/30-10-ess-relevant-consensus.csv"

pref_df = pd.read_csv(preference_filename)
act_df = pd.read_csv(actions_filename)
cons_df = pd.merge(pref_df, act_df, on='p')
cons_df['p'] = cons_df['p'].round(1)
final_df = pd.DataFrame()
for consensus in relevent_consensuses:
    filtered_df = cons_df[cons_df['p'] == consensus][relevent_columns]
    filtered_df['decision'] = (filtered_df['Rel_div_p'] * filtered_df['Rel-Nonrel']) + (filtered_df['Nonrel_div_p'] * filtered_df['Nonrel-Rel'])
    final_df = pd.concat([final_df, filtered_df], ignore_index=True)
final_df.to_csv(final_df_savename)

#### Find the difference between each agent and each of the 4 different strategies (only on decision)

For every agent that has a value system that we have just generated, we now compare the agents decision against the different decisions found by each strategy.

In [37]:
agent_df = df
strategy_df = final_df
agent_df['min_distance'] = agent_df.apply(
    lambda row: min(abs(row['decision'] - strategy_decision) for strategy_decision in strategy_df['decision']),
    axis=1
)

agent_df['closest_p'] = agent_df.apply(
    lambda row: strategy_df.loc[(strategy_df['decision'] - row['decision']).abs().idxmin(), 'p'],
    axis=1
)

agent_df

print(agent_df['closest_p'].value_counts())



closest_p
1.5     11
1.0      7
10.0     3
4.0      1
1.1      1
Name: count, dtype: int64
